In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import mpl_toolkits.mplot3d.axes3d as p3
%matplotlib notebook

In [2]:
def animate_wave_simulation_2D(U, dt):
    """
    Creates a 2D animation of a wave simulation.

    Automatically selects color-scale.  In the event of instability or
    a wild range of values, some components might not be easily seen.
    Consider tweaking this scale if you need to.

    Parameters
    ----------
    U: list-like
       A list of 2D wavefields to animate.

    Returns
    -------
    Matplotlib animation class instance.

    """

    fig, ax = plt.subplots()

    cmin, cmax = U[0].min(), U[0].max()
    for u in U:
        cmin = min(cmin, u.min())
        cmax = max(cmax, u.max())

    cmin = max(-1, cmin)
    cmax = min(1, cmax)

    time_text = ax.set_title('Current Time:')
    
    im = plt.imshow(U[0], clim=(cmin,cmax), cmap='gray')
    def animate(i):
        im.set_data(U[i])
        time_text.set_text('Current Time = {:.3f}'.format(i * dt))
        return (im,)
    plt.xlabel('x')
    plt.ylabel('y')
    ani = animation.FuncAnimation(fig, animate, interval=50, repeat_delay=1000)

    return ani

In [2]:
def animate_wave_simulation_3D(U, dt):
    """
    Creates a 3D surface animation of a wave simulation.

    Automatically selects color-scale.  In the event of instability or
    a wild range of values, some components might not be easily seen.
    Consider tweaking this scale if you need to.

    Adapted from https://stackoverflow.com/a/45713451 under the 
    CC BY-SA 3.0 license.

    Parameters
    ----------
    U: list-like
       A list of 2D wavefields to animate.

    Returns
    -------
    Matplotlib animation class instance.

    """
    
    grid_y = np.linspace(0, 1, U[0].shape[0])
    grid_x = np.linspace(0, 1, U[0].shape[1])
    YY, XX = np.meshgrid(grid_y, grid_x)

    fig = plt.figure()
    ax = p3.Axes3D(fig)


    cmin, cmax = U[0].min(), U[0].max()
    for u in U:
        cmin = min(cmin, u.min())
        cmax = max(cmax, u.max())

    cmin = max(-1, cmin)
    cmax = min(1, cmax)

    surf = [ax.plot_surface(YY, XX, U[0], cmap="viridis", clim=(cmin, cmax))]
    ax.set_zlim(1.1*cmin, 1.1*cmax)

    time_text = ax.text2D(0.25, 0.95, 'Time:', horizontalalignment='center',
                        verticalalignment='center', transform=ax.transAxes)
    
    def animate(i, U, surf):
        surf[0].remove()
        surf[0] = ax.plot_surface(YY, XX, U[i], cmap="viridis", clim=(-1,1))
        time_text.set_text('Current Time = {:.3f}'.format(i * dt))
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    
    ani = animation.FuncAnimation(fig, animate, fargs=(U, surf), interval=50, repeat_delay=1000)

    return ani

In [3]:
def wave_function(t, mx, my, x, y):
    """
    Solves the Standing Wave equation at certain spatial and time points
    
    Parameters
    ----------
    t: float
       The current time to solve the equation at
    mx: integer
        Number of standing wave nodes in the x dimension
    my: integer
        Number of standing wave nodes in the y dimension
    x: float
        The current point on the grid in the x dimension
    y: float
        The current point on the grid in the y dimension

    Returns
    -------
    float
        Solution to the wave equation at the specific spatial and time step
    """
    
    w = np.pi * (np.sqrt((mx ** 2) + (my ** 2)))
    return np.sin(mx * x * np.pi) * np.sin(my * y * np.pi) * np.cos(w * t)

In [4]:
def initial_conditions(xgrid, ygrid, mx, my, n, dt):
    """
    Sets the inital conditions for the simulation by solving for u^0 and u^{-deltat}
    
    Parameters
    ----------
    xgrid: numpy.ndarray
       The grid of possible values in the x dimension
    ygrid: ndarray
       The grid of possible values in the y dimension
    mx: integer
        Number of standing wave nodes in the x dimension
    my: integer
        Number of standing wave nodes in the y dimension
    n: integer
        The number of grid points in both the x and y dimension (n for x and y dimension)
    dt: float
        Step size for the time

    Returns
    -------
    u_k: numpy.ndarray
        2D array for u^0
    u_kmin1: numpy.ndarray
        2D array for u^{-deltat}
    """
    
    # Creates the empty 2D arrays
    u_k = np.zeros([n, n])
    u_kmin1 = np.zeros([n, n])
    
    for j in range(n):
        for i in range(n):
            u_k[j,i] = wave_function(0, mx, my, xgrid[i], ygrid[j])
            u_kmin1[j,i] = wave_function(dt, mx, my, xgrid[i], ygrid[j])
    return u_k, u_kmin1

In [5]:
def grid_solver(xgrid, ygrid, mx, my, n, t):
"""
    Solves the true solution to the standing wave equation at a certain timestep over the entire grid.
    
    Parameters
    ----------
    xgrid: numpy.ndarray
       The grid of possible values in the x dimension
    ygrid: ndarray
       The grid of possible values in the y dimension
    mx: integer
        Number of standing wave nodes in the x dimension
    my: integer
        Number of standing wave nodes in the y dimension
    n: integer
        The number of grid points in both the x and y dimension (n for x and y dimension)
    t: float
        Current timestep

    Returns
    -------
    u_k: numpy.ndarray
        2D array for u^t 
    """

    u_k = np.zeros([n, n])
    
    for j in range(n):
        for i in range(n):
            u_k[j,i] = wave_function(t, mx, my, xgrid[i], ygrid[j])
    return u_k

In [6]:
def time_step(u_kmin1, u_k, n, dt):
"""
    Uses the 5-point stencil method to simulate the solution to the wave equation for one step forward in time.
    
    Parameters
    ----------
    u_kmin1: numpy.ndarray
       The grid solution at the previous timstep
    u_k: numpy.ndarray
       The grid solution at the current timstep
    n: integer
        The number of grid points in both the x and y dimension (n for x and y dimension)
    dt: float
        Step size for the time

    Returns
    -------
    u_plus1: numpy.ndarray
        Solution for the next timestep, u^{kplus1}
    """
    deltax = (1 - 0) / (n - 1)
    # Create empty grids to hold solutions
    u_kplus1 = np.zeros([n, n])
    l_kplus1 = np.zeros([n, n])
    
    # Loop over all points on the grid
    for j in range(n):
        for i in range(n):
            # If add border point replace with a 0 (Effectively breaking)
            if (j == 0 or i == 0) or (j == (n - 1) or i == (n - 1)):
                u_kplus1[j, i] = 0
            # If not at border point
            else:
                # Compute laplacian
                l_kplus1[j, i] = (-4*u_k[j,i] + u_k[j-1,i] + u_k[j,i-1] + u_k[j+1,i] + u_k[j,i+1]) / (deltax ** 2)
                # Compute next time step solution using the laplacian and previous solutions
                u_kplus1[j, i] = -u_kmin1[j, i] + 2*u_k[j, i] + (dt**2)*l_kplus1[j, i]
    return u_kplus1

In [7]:
def simulation(T, n, mx, my, alpha):
"""
    Uses the 5-point stencil method to simulate the solution to the wave equation for one step forward in time.
    
    Parameters
    ----------
    T: Real number
       The final point to end the simulation at.
    n: integer
        The number of grid points in both the x and y dimension (n for x and y dimension)
    dt: float
        Step size for the time
    mx: integer
        Number of standing wave nodes in the x dimension
    my: integer
        Number of standing wave nodes in the y dimension
    alpha: float
        Safetey paramater. Controls the stability of the system, alpha <= 1.0 is stable.

    Returns
    -------
    U: list-like
        A list of the solutions to the wave equation over all timesteps. 
    dt: float
        The step size for time. Returned for ease of animating.
    """
    #Compute dx, then create grids in x and y dimension
    deltax = (1 - 0) / (n - 1)
    x_grid = np.linspace(0, 1, n)
    y_grid = np.linspace(0, 1, n)
    
    #Compute deltat and nt
    deltat = (alpha * deltax) / (np.sqrt(2))
    nt = (T - 0) / (deltat)
    
    # Creates the initial conditions for the simulation
    u_k, u_kmin1 = initial_conditions(x_grid, y_grid, mx, my, n, -deltat)
    u_kplus1 = np.zeros([n, n])
    
    # Adds the initial conditions to the overall list of solutions
    U = []
    U.append(u_kmin1)
    U.append(u_k)
    
    # Loop over all timesteps
    for i in range(int(nt)):
        # Solve one time step of the simulation
        u_kplus1 = time_step(u_kmin1, u_k, n, deltat)
        U.append(u_kplus1)
        # Swap the members for the next timestep
        u_prev, u_curr, u_next = u_curr, u_next, u_prev
        
    return U, deltat